In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import sklearn
import joblib
import time
from string import punctuation

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score, plot_roc_curve, make_scorer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight
import lightgbm as lgb
import nltk
from nltk.corpus import stopwords

tqdm.pandas()

In [ ]:
selected_features_1 =["Live Birth Occurrence","Patient Age at Treatment","Total Number of Previous cycles, Both IVF and DI", "Total number of IVF pregnancies","Total number of live births - conceived through IVF","Type of Infertility - Female Primary",
"Type of Infertility - Female Secondary", "Type of Infertility - Male Primary","Type of Infertility - Male Secondary","Type of Infertility -Couple Primary","Type of Infertility -Couple Secondary",
"Cause  of Infertility - Tubal disease", "Cause of Infertility - Ovulatory Disorder","Cause of Infertility - Male Factor","Cause of Infertility - Patient Unexplained","Cause of Infertility - Endometriosis",
"Cause of Infertility - Cervical factors","Cause of Infertility - Female Factors","Cause of Infertility - Partner Sperm Concentration","Cause of Infertility -  Partner Sperm Morphology","Causes of Infertility - Partner Sperm Motility","Cause of Infertility -  Partner Sperm Immunological factors",
"Stimulation used","Egg Source","Sperm From","Fresh Cycle","Frozen Cycle","Eggs Mixed With Partner Sperm","Eggs Thawed","Embryos Transfered","Fresh Eggs Collected"]

selected_features_2 =["Live Birth Occurrence","Patient Age at Treatment", "Total Number of Previous treatments, Both IVF and DI at clinic", "Total Number of Previous IVF cycles","Total number of IVF pregnancies","Type of Infertility - Female Primary",
"Type of Infertility - Female Secondary", "Type of Infertility - Male Primary","Type of Infertility - Male Secondary","Type of Infertility -Couple Primary","Type of Infertility -Couple Secondary",
"Cause  of Infertility - Tubal disease", "Cause of Infertility - Ovulatory Disorder","Cause of Infertility - Male Factor","Cause of Infertility - Patient Unexplained","Cause of Infertility - Endometriosis",
"Cause of Infertility - Cervical factors","Cause of Infertility - Female Factors","Cause of Infertility - Partner Sperm Concentration","Cause of Infertility -  Partner Sperm Morphology","Causes of Infertility - Partner Sperm Motility","Cause of Infertility -  Partner Sperm Immunological factors",
"Stimulation used","Embryos Transfered","Type of treatment - IVF or DI","Specific treatment type","Total Embryos Created" ]

joblib.dump(selected_features_1, "E:\\projects\\plds_teamc\\data\\selected_features_1.pkl")
joblib.dump(selected_features_2, "E:\\projects\\plds_teamc\\data\\selected_features_2.pkl")

In [ ]:





dataset_1 = dataset_1.dropna(subset = ["Egg Source","Sperm From"])
dataset_1["Live Birth Occurrence"]=dataset_1["Live Birth Occurrence"].fillna(0)
dataset_1 = dataset_1[dataset_1['Stimulation used'] != 0]
#Convert patient age to categorical data
dataset_1 = dataset_1[dataset_1['Patient Age at Treatment'] != "999"]
dataset_1["Patient Age at Treatment"] = dataset_1["Patient Age at Treatment"].str.replace(" ","")
dataset_1["Patient Age at Treatment"] = dataset_1.replace({"Patient Age at Treatment":{"18-34":"0", "35-37":"1" ,"38-39": "2","40-42": "3", "43-44": "4","44-50": "5"}})
dataset_1 = dataset_1.replace("> 50","50")
dataset_1 = dataset_1.replace(">=5","5")


dataset_1 = dataset_1.replace("Patient","1")
dataset_1 = dataset_1.replace("Partner","1")
dataset_1 = dataset_1.replace("Donor","0")
dataset_1 = dataset_1.replace("Partner & Donor","0")
dataset_1 = dataset_1[dataset_1["Sperm From"].str.contains("not assigned")==False]

dataset_1_X = dataset_1.loc[:, dataset_1.columns != "Live Birth Occurrence"]
dataset_1_Y = dataset_1["Live Birth Occurrence"]

X_train_1, X_test_1, Y_train_1, Y_test_1 = train_test_split(dataset_1_X, dataset_1_Y, test_size=0.3, random_state=42)

In [ ]:
def main_load(file_loc1, file_loc2, feature_loc):
    df1 = pd.read_excel(file_loc1,engine='pyxlsb')
    df2 = pd.read_excel(file_loc2,engine='pyxlsb')
    features = joblib.load(feature_loc)
    
    df_combined = pd.concat([df1, df2])
    dataset = df_combined[features]





    x_all, y_all = split_xy(df, ['comment_text'], ['toxic'])
    x_train, y_train,x_valid, y_valid,x_test, y_test = run_split_data(x_all, y_all, 'toxic', 0.2)
    joblib.dump(x_train, "E:\\projects\\plds_latihan\\pipeline\\x_train.pkl")
    joblib.dump(y_train, "E:\\projects\\plds_latihan\\pipeline\\y_train.pkl")
    joblib.dump(x_valid, "E:\\projects\\plds_latihan\\pipeline\\x_valid.pkl")
    joblib.dump(y_valid, "E:\\projects\\plds_latihan\\pipeline\\y_valid.pkl")
    joblib.dump(x_test, "E:\\projects\\plds_latihan\\pipeline\\x_test.pkl")
    joblib.dump(y_test, "E:\\projects\\plds_latihan\\pipeline\\y_test.pkl")

    return x_train, y_train, x_valid, y_valid, x_test, y_test

In [ ]:
file_loc1 = 'E:/projects/plds_teamc/data/ar-2010-2014-xlsb.xlsb'
file_loc2 = 'E:/projects/plds_teamc/data/ar-2015-2016-xlsb.xlsb'
feature_loc = 'E:\\projects\\plds_teamc\\data\\selected_features_1.pkl'


x_train, y_train, x_valid, y_valid, x_test, y_test = main_load(file_loc1,file_loc2, feature)
